In [ ]:
#!pip install keras==2.15.0 matplotlib pillow numpy tensorflow 
import tensorflow as tf
import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *

X = []
y = []
#Randomly generate 10 sample arrays for X and y where y is the flip of X
for i in range(10):
  X.append(np.round(np.random.rand(8,8)).reshape(8,8,1))
  y.append(np.abs(1-X[-1]).reshape(8,8,1))

X = np.array(X)
y = np.array(y)
input_size = X[0].shape
inputs = Input(input_size)  
print(input_size)
print(inputs)

#Construct the 1st layer with 2 filters of size 3x3, ReLU as activation function, and padding
conv1 = Conv2D(filters = 2,   
               kernel_size = 3,  
               activation = 'relu',   
               padding = 'same',     
               kernel_initializer = 'he_normal')(inputs)
conv1

In [ ]:
#Construct the second layer with two filters of size 3x3x2. 
#The double of the number of channels can be handled by the function so no extra parameter is needed.
conv2 = Conv2D(filters = 2,  
               kernel_size = 3,  
               activation = 'relu',    
               padding = 'same',    
               kernel_initializer = 'he_normal')(conv1)

#Apply max pooling
pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv2, pool1

In [ ]:
#Construct the 3rd and 4th layer
conv3 = Conv2D(filters = 3, kernel_size = 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv4 = Conv2D(filters = 3, kernel_size = 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)

#Apply upsampling to the output of the 4th layer and use the result as the input of the 5th layer
up1 = UpSampling2D(size = (2,2))(conv4)
conv5 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(up1)

#Concatenate the 2nd and the 5th layer, stacking the images on the third axis
concat1 = concatenate([conv2,conv5], axis = 3)

#Show the result for upsampling and concatenation
up1, concat1

In [ ]:
#Construct the 6th and output layer
conv6 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(concat1)
out = Conv2D(1, 3, activation = 'sigmoid', padding = 'same', kernel_initializer = 'he_normal')(conv6)

#Create the model, show its summary and compile it with loss and metrics parameters
model = Model(inputs, out)
model.summary()
model.compile(loss = 'mse', metrics = ['BinaryIoU'])

#Fit the data and train it for 100 epochs
model.fit(X, y, epochs=100) 

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

im = Image.open(os.getcwd()+"/1.png")
im_matrix = np.array(im)
print(im_matrix,im_matrix.shape)

#Get the flipped image array
flip = np.abs(1-im_matrix)

#Predict the output with the model
#keras needs the shape to match the input shape it expected so it reshapes twice
predicted = model.predict(im_matrix.reshape(1,8,8,1)).reshape(8,8)   

#Display the three images
fig = plt.figure(figsize=(15,5))
ax1,ax2,ax3 = fig.subplots(1,3)
ax1.imshow(im_matrix), ax1.set_title("Original")
ax2.imshow(flip), ax2.set_title("Flipped")
ax3.imshow(predicted), ax3.set_title("Predicted Output")